In [1]:
# =============================================================================
# Import OHLCV data and transform it to Renko
# Author : Mayank Rasu (http://rasuquant.com/wp/)

# Please report bug/issues in the Q&A section
# =============================================================================

# Import necesary libraries
import yfinance as yf
from stocktrends import Renko

# Download historical data for required stocks
tickers = ["AMZN","GOOG","MSFT"]
ohlcv_data = {}
hour_data = {}
renko_data = {}

# looping over tickers and storing OHLCV dataframe in dictionary
for ticker in tickers:
    temp = yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how="any",inplace=True)
    ohlcv_data[ticker] = temp
    temp = yf.download(ticker,period='1y',interval='1h')
    temp.dropna(how="any",inplace=True)
    hour_data[ticker] = temp
    
def ATR(DF, n=14):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1))
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].ewm(com=n, min_periods=n).mean()
    return df["ATR"]

def renko_DF(DF, hourly_df):
    "function to convert ohlc data into renko bricks"
    df = DF.copy()
    df.reset_index(inplace=True)
    df.drop("Close",axis=1,inplace=True)
    df.columns = ["date","open","high","low","close","volume"]
    df2 = Renko(df)
    df2.brick_size = 3*round(ATR(hourly_df,120).iloc[-1],0)
    renko_df = df2.get_ohlc_data() #if using older version of the library please use get_bricks() instead
    return renko_df

for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker],hour_data[ticker])

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
ohlcv_data

{'AMZN':                                  Open        High         Low       Close  \
 Datetime                                                                    
 2024-04-23 10:55:00-04:00  177.859695  178.199997  177.824997  178.100006   
 2024-04-23 11:00:00-04:00  178.110001  178.149994  177.320007  177.444702   
 2024-04-23 11:05:00-04:00  177.449997  177.630005  177.410004  177.559998   
 2024-04-23 11:10:00-04:00  177.589996  178.139999  177.589996  178.110107   
 2024-04-23 11:15:00-04:00  178.130005  178.330002  178.029999  178.139999   
 ...                               ...         ...         ...         ...   
 2024-05-23 10:35:00-04:00  182.740097  183.233002  182.740097  183.220001   
 2024-05-23 10:40:00-04:00  183.219894  183.304993  182.910004  182.945007   
 2024-05-23 10:45:00-04:00  182.964996  183.349197  182.940094  183.300003   
 2024-05-23 10:50:00-04:00  183.300003  183.330002  182.759995  182.761307   
 2024-05-23 10:55:00-04:00  182.759995  182.832993  182.

In [8]:
renko_data['AMZN']

,date,open,high,low,close,uptrend
0,2024-04-23 10:55:00-04:00,174.0,177.0,174.0,177.0,True
1,2024-04-24 09:35:00-04:00,177.0,180.0,177.0,180.0,True
2,2024-04-25 09:30:00-04:00,177.0,177.0,174.0,174.0,False
3,2024-04-25 09:30:00-04:00,174.0,174.0,171.0,171.0,False
4,2024-04-26 09:35:00-04:00,174.0,177.0,174.0,177.0,True
5,2024-04-26 12:35:00-04:00,177.0,180.0,177.0,180.0,True
6,2024-05-01 14:40:00-04:00,180.0,183.0,180.0,183.0,True
7,2024-05-03 09:30:00-04:00,183.0,186.0,183.0,186.0,True
8,2024-05-07 09:40:00-04:00,186.0,189.0,186.0,189.0,True
9,2024-05-21 09:30:00-04:00,186.0,186.0,183.0,183.0,False


In [5]:
import pandas as pd

{'AMZN':                         date   open   high    low  close  uptrend
 0  2024-04-23 10:55:00-04:00  174.0  177.0  174.0  177.0     True
 1  2024-04-24 09:35:00-04:00  177.0  180.0  177.0  180.0     True
 2  2024-04-25 09:30:00-04:00  177.0  177.0  174.0  174.0    False
 3  2024-04-25 09:30:00-04:00  174.0  174.0  171.0  171.0    False
 4  2024-04-26 09:35:00-04:00  174.0  177.0  174.0  177.0     True
 5  2024-04-26 12:35:00-04:00  177.0  180.0  177.0  180.0     True
 6  2024-05-01 14:40:00-04:00  180.0  183.0  180.0  183.0     True
 7  2024-05-03 09:30:00-04:00  183.0  186.0  183.0  186.0     True
 8  2024-05-07 09:40:00-04:00  186.0  189.0  186.0  189.0     True
 9  2024-05-21 09:30:00-04:00  186.0  186.0  183.0  183.0    False,
 'GOOG':                          date   open   high    low  close  uptrend
 0   2024-04-23 10:55:00-04:00  156.0  159.0  156.0  159.0     True
 1   2024-04-26 09:30:00-04:00  159.0  162.0  159.0  162.0     True
 2   2024-04-26 09:30:00-04:00  162.0  165

In [1]:
import talib

In [2]:

?talib.BOP

Signature: talib.BOP(open, high, low, close)
Docstring:
BOP(open, high, low, close)

Balance Of Power (Momentum Indicators)

Inputs:
    prices: ['open', 'high', 'low', 'close']
Outputs:
    real
Type:      function

In [3]:
dir(talib)

['ACOS',
 'AD',
 'ADD',
 'ADOSC',
 'ADX',
 'ADXR',
 'APO',
 'AROON',
 'AROONOSC',
 'ASIN',
 'ATAN',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BETA',
 'BOP',
 'CCI',
 'CDL2CROWS',
 'CDL3BLACKCROWS',
 'CDL3INSIDE',
 'CDL3LINESTRIKE',
 'CDL3OUTSIDE',
 'CDL3STARSINSOUTH',
 'CDL3WHITESOLDIERS',
 'CDLABANDONEDBABY',
 'CDLADVANCEBLOCK',
 'CDLBELTHOLD',
 'CDLBREAKAWAY',
 'CDLCLOSINGMARUBOZU',
 'CDLCONCEALBABYSWALL',
 'CDLCOUNTERATTACK',
 'CDLDARKCLOUDCOVER',
 'CDLDOJI',
 'CDLDOJISTAR',
 'CDLDRAGONFLYDOJI',
 'CDLENGULFING',
 'CDLEVENINGDOJISTAR',
 'CDLEVENINGSTAR',
 'CDLGAPSIDESIDEWHITE',
 'CDLGRAVESTONEDOJI',
 'CDLHAMMER',
 'CDLHANGINGMAN',
 'CDLHARAMI',
 'CDLHARAMICROSS',
 'CDLHIGHWAVE',
 'CDLHIKKAKE',
 'CDLHIKKAKEMOD',
 'CDLHOMINGPIGEON',
 'CDLIDENTICAL3CROWS',
 'CDLINNECK',
 'CDLINVERTEDHAMMER',
 'CDLKICKING',
 'CDLKICKINGBYLENGTH',
 'CDLLADDERBOTTOM',
 'CDLLONGLEGGEDDOJI',
 'CDLLONGLINE',
 'CDLMARUBOZU',
 'CDLMATCHINGLOW',
 'CDLMATHOLD',
 'CDLMORNINGDOJISTAR',
 'CDLMORNINGSTAR',
 'CDLONNECK',
 'CD

In [6]:
talib.get_function_groups()['Pattern Recognition']

['CDL2CROWS',
 'CDL3BLACKCROWS',
 'CDL3INSIDE',
 'CDL3LINESTRIKE',
 'CDL3OUTSIDE',
 'CDL3STARSINSOUTH',
 'CDL3WHITESOLDIERS',
 'CDLABANDONEDBABY',
 'CDLADVANCEBLOCK',
 'CDLBELTHOLD',
 'CDLBREAKAWAY',
 'CDLCLOSINGMARUBOZU',
 'CDLCONCEALBABYSWALL',
 'CDLCOUNTERATTACK',
 'CDLDARKCLOUDCOVER',
 'CDLDOJI',
 'CDLDOJISTAR',
 'CDLDRAGONFLYDOJI',
 'CDLENGULFING',
 'CDLEVENINGDOJISTAR',
 'CDLEVENINGSTAR',
 'CDLGAPSIDESIDEWHITE',
 'CDLGRAVESTONEDOJI',
 'CDLHAMMER',
 'CDLHANGINGMAN',
 'CDLHARAMI',
 'CDLHARAMICROSS',
 'CDLHIGHWAVE',
 'CDLHIKKAKE',
 'CDLHIKKAKEMOD',
 'CDLHOMINGPIGEON',
 'CDLIDENTICAL3CROWS',
 'CDLINNECK',
 'CDLINVERTEDHAMMER',
 'CDLKICKING',
 'CDLKICKINGBYLENGTH',
 'CDLLADDERBOTTOM',
 'CDLLONGLEGGEDDOJI',
 'CDLLONGLINE',
 'CDLMARUBOZU',
 'CDLMATCHINGLOW',
 'CDLMATHOLD',
 'CDLMORNINGDOJISTAR',
 'CDLMORNINGSTAR',
 'CDLONNECK',
 'CDLPIERCING',
 'CDLRICKSHAWMAN',
 'CDLRISEFALL3METHODS',
 'CDLSEPARATINGLINES',
 'CDLSHOOTINGSTAR',
 'CDLSHORTLINE',
 'CDLSPINNINGTOP',
 'CDLSTALLEDPATTERN',
 

In [8]:
import yfinance as yf

In [9]:
data = yf.download("RELIANCE.NS",start="2020-01-01",end="2024-06-10")

[*********************100%%**********************]  1 of 1 completed


In [10]:
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,1387.957031,1396.277466,1376.527954,1380.276611,1360.645142,7002234
2020-01-02,1382.471069,1408.941040,1382.471069,1403.775024,1383.809326,8855158
2020-01-03,1401.671997,1409.581055,1392.528687,1405.466553,1385.476807,10492349
2020-01-06,1389.785767,1397.008911,1369.670410,1372.870605,1353.344482,12259588
2020-01-07,1388.871338,1403.043579,1383.842529,1393.991699,1374.165283,8341811
...,...,...,...,...,...,...
2024-06-03,2966.000000,3029.000000,2918.000000,3020.649902,3020.649902,10763971
2024-06-04,2996.100098,2996.100098,2718.600098,2794.550049,2794.550049,18354549
2024-06-05,2844.000000,2862.300049,2768.199951,2841.500000,2841.500000,8732445


In [13]:
arron=talib.AROONOSC(data['High'],data['Close'],timeperiod=14)
data['ARRON_INDICATOR'] = arron

In [16]:
RSI = talib.RSI(data["Close"],timeperiod=14)
data["RSI"] =RSI

In [17]:
data

,Open,High,Low,Close,Adj Close,Volume,ARRON_INDICATOR,RSI
Date,,,,,,,,
2020-01-01,1387.957031,1396.277466,1376.527954,1380.276611,1360.645142,7002234,NaN,NaN
2020-01-02,1382.471069,1408.941040,1382.471069,1403.775024,1383.809326,8855158,NaN,NaN
2020-01-03,1401.671997,1409.581055,1392.528687,1405.466553,1385.476807,10492349,NaN,NaN
2020-01-06,1389.785767,1397.008911,1369.670410,1372.870605,1353.344482,12259588,NaN,NaN
2020-01-07,1388.871338,1403.043579,1383.842529,1393.991699,1374.165283,8341811,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-06-03,2966.000000,3029.000000,2918.000000,3020.649902,3020.649902,10763971,100.000000,63.690969
2024-06-04,2996.100098,2996.100098,2718.600098,2794.550049,2794.550049,18354549,-7.142857,42.342948
2024-06-05,2844.000000,2862.300049,2768.199951,2841.500000,2841.500000,8732445,-7.142857,46.363277


In [19]:
data["ARRON_INDICATOR"].value_counts()

ARRON_INDICATOR
 78.571429     69
 100.000000    63
-78.571429     61
-42.857143     58
 64.285714     58
-35.714286     53
 42.857143     53
 57.142857     53
-64.285714     52
 92.857143     51
 85.714286     51
 50.000000     48
-50.000000     47
-57.142857     46
 71.428571     40
-28.571429     38
-71.428571     37
 28.571429     36
-85.714286     30
-100.000000    25
 35.714286     24
-92.857143     23
-21.428571     15
 21.428571     13
 14.285714     12
-7.142857      11
-14.285714      8
 7.142857       5
 0.000000       4
Name: count, dtype: int64

In [20]:
?talib.AROONOSC

Signature: talib.AROONOSC(high, low, timeperiod=-2147483648)
Docstring:
AROONOSC(high, low[, timeperiod=?])

Aroon Oscillator (Momentum Indicators)

Inputs:
    prices: ['high', 'low']
Parameters:
    timeperiod: 14
Outputs:
    real
Type:      function

### Using Abstract API

In [21]:
from talib import abstract

In [22]:
data

,Open,High,Low,Close,Adj Close,Volume,ARRON_INDICATOR,RSI
Date,,,,,,,,
2020-01-01,1387.957031,1396.277466,1376.527954,1380.276611,1360.645142,7002234,NaN,NaN
2020-01-02,1382.471069,1408.941040,1382.471069,1403.775024,1383.809326,8855158,NaN,NaN
2020-01-03,1401.671997,1409.581055,1392.528687,1405.466553,1385.476807,10492349,NaN,NaN
2020-01-06,1389.785767,1397.008911,1369.670410,1372.870605,1353.344482,12259588,NaN,NaN
2020-01-07,1388.871338,1403.043579,1383.842529,1393.991699,1374.165283,8341811,NaN,NaN
...,...,...,...,...,...,...,...,...
2024-06-03,2966.000000,3029.000000,2918.000000,3020.649902,3020.649902,10763971,100.000000,63.690969
2024-06-04,2996.100098,2996.100098,2718.600098,2794.550049,2794.550049,18354549,-7.142857,42.342948
2024-06-05,2844.000000,2862.300049,2768.199951,2841.500000,2841.500000,8732445,-7.142857,46.363277


In [28]:
data.rename(columns = {"High":"high","Open":"open","Close":"close","Volume":'volume',"Low":"low"},inplace=True)

In [30]:

engulfing=abstract.CDLENGULFING(data)

In [31]:
data['Engulfing'] = engulfing

In [32]:
data

,open,high,low,close,Adj Close,volume,ARRON_INDICATOR,RSI,Engulfing
Date,,,,,,,,,
2020-01-01,1387.957031,1396.277466,1376.527954,1380.276611,1360.645142,7002234,NaN,NaN,0
2020-01-02,1382.471069,1408.941040,1382.471069,1403.775024,1383.809326,8855158,NaN,NaN,0
2020-01-03,1401.671997,1409.581055,1392.528687,1405.466553,1385.476807,10492349,NaN,NaN,0
2020-01-06,1389.785767,1397.008911,1369.670410,1372.870605,1353.344482,12259588,NaN,NaN,0
2020-01-07,1388.871338,1403.043579,1383.842529,1393.991699,1374.165283,8341811,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
2024-06-03,2966.000000,3029.000000,2918.000000,3020.649902,3020.649902,10763971,100.000000,63.690969,0
2024-06-04,2996.100098,2996.100098,2718.600098,2794.550049,2794.550049,18354549,-7.142857,42.342948,0
2024-06-05,2844.000000,2862.300049,2768.199951,2841.500000,2841.500000,8732445,-7.142857,46.363277,0


In [33]:
data['Engulfing'].value_counts()

Engulfing
 0      1015
-100      56
 100      27
Name: count, dtype: int64